### Implementing simple Chatbot Using LangGraph

In [ ]:
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END

## Reducers
from typing import Annotated
from langgraph.graph.message import add_messages

In [ ]:
class State(TypedDict):
    messages:Annotated[list,add_messages]

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

# Check if API keys are loaded
openai_key = os.getenv("OPENAI_API_KEY")
groq_key = os.getenv("GROQ_API_KEY")

if openai_key:
    os.environ["OPENAI_API_KEY"] = openai_key
    print("OpenAI API key loaded")
else:
    print("OpenAI API key not found")

if groq_key:
    os.environ["GROQ_API_KEY"] = groq_key
    print("Groq API key loaded")
else:
    print("Groq API key not found")

In [ ]:
# Only run if OpenAI key is available
if openai_key:
    from langchain_openai import ChatOpenAI
    llm=ChatOpenAI(model="gpt-4o")
    print(llm.invoke("Hello"))
else:
    print("Skipping OpenAI test - no API key")

In [ ]:
# Setup Groq with proper error handling
if groq_key:
    from langchain_groq import ChatGroq
    
    # Use a valid Groq model
    llm_groq = ChatGroq(model="llama-3.1-8b-instant", groq_api_key=groq_key)
    print(llm_groq.invoke("Hey I am John and i like to play cricket"))
else:
    print("Cannot initialize Groq - API key not found")
    print("Please add GROQ_API_KEY to your .env file")

### We Will start With Creating Nodes

In [ ]:
from langchain_core.messages import HumanMessage

def superbot(state: State):
    if 'llm_groq' in globals():
        return {"messages": [llm_groq.invoke(state['messages'])]}
    else:
        return {"messages": [HumanMessage(content="Error: Groq API not available")]}

In [ ]:
# Only create graph if Groq is available
if groq_key and 'llm_groq' in globals():
    graph = StateGraph(State)
    
    ## node
    graph.add_node("SuperBot", superbot)
    ## Edges
    
    graph.add_edge(START, "SuperBot")
    graph.add_edge("SuperBot", END)
    
    graph_builder = graph.compile()
    
    ## Display
    from IPython.display import Image, display
    display(Image(graph_builder.get_graph().draw_mermaid_png()))
else:
    print("Cannot create graph - Groq API not available")

In [ ]:
## Invocation
if groq_key and 'graph_builder' in globals():
    from langchain_core.messages import HumanMessage
    
    result = graph_builder.invoke({'messages': [HumanMessage(content="Hi, My name is John And I like cricket")]})
    print(result)
else:
    print("Cannot invoke graph - setup incomplete")

#### Streaming The responses

In [ ]:
if groq_key and 'graph_builder' in globals():
    for event in graph_builder.stream({"messages": [HumanMessage(content="Hello My name is Krish")]}):
        print(event)
else:
    print("Cannot stream - setup incomplete")